In [1]:
from selenium import webdriver
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import csv
import json
from collections import Counter
import jwt
import datetime
import requests
from lxml import html
import re
import dask.dataframe as dd # pip install dask

NaN = np.nan


headers = {
    'User-Agent': 'Mie Damgaard',
    'mail' : 'rks516@alumni.ku.dk'
}

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
DST_df = pd.read_csv('CSV_output/DST.csv')
laeger_st_post_df = pd.read_csv('CSV_output/laeger_stationer_post.csv')
#postnummer_by_df = pd.read_csv('CSV_input/postnumre', encoding='ISO-8859-1', delimiter=';', header=None)
boligsiden_20000_df = pd.read_csv('../outputdata/df_final_3.csv')

In [8]:
boligsiden_20000_df = boligsiden_20000_df.rename(columns={
    'Municipiality':'Kommune'
})

DST_df['Kommune'] = DST_df['Kommune'].str.strip().str.lower()
boligsiden_20000_df['Kommune'] = boligsiden_20000_df['Kommune'].str.strip().str.lower()

In [10]:
merged_df = pd.merge(boligsiden_20000_df, DST_df, on ='Kommune', how = 'left')
merged_df

,Unnamed: 0,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Kommune,Rooms_imputed,Toilets_imputed,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,NaN,NaN,Rækkehus,D,Høj,24.60,25.00,1.0,128.0,388.0,2889.0,5.0,1947,1.0,2600,https://www.boligsiden.dk/addresses/0a3f50a4-b...,0,0,0,glostrup kommune,0,0,2660,1096,407100.0,47.4,23899.0,0.738859,0.055232,0.205908,13.3
1,"Strandpromenaden 21, 1. tv.,3000",6095000.0,Skolen i Bymidten,0.336,NaN,NaN,Ejerlejlighed,A2015,Lav,25.52,28.50,0.0,136.0,136.0,6056.0,5.0,2020,2.0,3000,https://www.boligsiden.dk/addresses/0d5cec39-0...,0,0,0,helsingør kommune,0,0,5089,1787,463089.0,58.8,64016.0,0.857786,0.045442,0.096773,23.6
2,"Hjortdalvej 331,9460",3995000.0,Skovsgård Tranum skole,5.240,6.9,3.6,Landejendom,D,Meget lav,25.70,32.85,0.0,173.0,173706.0,2089.0,5.0,1912,1.0,9460,https://www.boligsiden.dk/addresses/39ff92cd-f...,0,0,0,jammerbugt kommune,0,0,1421,537,409908.0,136.7,38170.0,0.928818,0.032119,0.039062,29.5
3,"Lystgårdsparken 34,8300",2798000.0,Parkvejens Skole,5.160,7.0,3.7,Villa,B,Mellem,25.10,30.32,180.0,159.0,758.0,1838.0,4.0,2008,1.0,8300,https://www.boligsiden.dk/addresses/363cc2db-6...,0,0,0,odder kommune,0,0,839,163,459467.0,70.8,23971.0,0.912770,0.036085,0.051145,23.6
4,"Fuglsøvej 92,8420",6000000.0,Molsskolen,3.740,7.4,3.7,Villa,D,Høj,25.90,33.80,0.0,265.0,8889.0,3434.0,12.0,1884,4.0,8420,https://www.boligsiden.dk/addresses/0a3f50bf-6...,0,0,0,syddjurs kommune,0,0,2009,647,432820.0,105.3,44033.0,0.916472,0.042582,0.040947,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19846,"Trongårdsparken 91,2800",10495000.0,Trongårdsskolen,0.242,8.0,3.8,Villa,D,Lav,24.38,21.49,109.0,184.0,1234.0,7030.0,6.0,1963,3.0,2800,https://www.boligsiden.dk/addresses/0a3f50a6-6...,0,0,0,lyngby-taarbæk kommune,0,0,4445,1234,576034.0,39.9,58530.0,0.808474,0.072612,0.118913,14.8
19847,"Alpinavej 16,8660",3995000.0,Stilling Skole,0.642,7.5,3.7,Rækkehus,C,Mellem,26.00,24.66,505.0,126.0,490.0,3253.0,4.0,1978,2.0,8660,https://www.boligsiden.dk/addresses/0a3f50c2-2...,0,0,0,skanderborg kommune,0,0,2406,774,528174.0,67.6,65376.0,0.920277,0.027365,0.052359,25.6
19848,"Thorsvej 39,9700",2375000.0,Skoledistrikt Vest,0.708,NaN,NaN,Villa,D,Mellem,26.30,31.23,290.0,186.0,1064.0,2230.0,7.0,1978,2.0,9700,https://www.boligsiden.dk/addresses/0a3f50c6-f...,0,0,0,brønderslev kommune,0,0,1720,933,410545.0,87.2,36602.0,0.916808,0.029097,0.054095,29.4
19849,"Strandjægervej 21,3630",4498000.0,Jægerspris Skole,10.380,6.8,3.6,Villa,A2010,Mellem,25.80,32.50,110.0,322.0,3579.0,4805.0,8.0,1787,2.0,3630,https://www.boligsiden.dk/addresses/0a3f50aa-4...,0,0,0,frederikssund kommune,0,0,2605,905,451448.0,51.8,46856.0,0.896683,0.035470,0.067846,24.7


In [12]:
laeger_st_post_df = pd.read_csv('CSV_output/laeger_stationer_post.csv')
laeger_st_post_df

merged_df = pd.merge(merged_df, laeger_st_post_df, on = 'Postal code', how = 'left')
merged_df['Antal_laeger'] = merged_df['Antal_laeger'].fillna(0)
merged_df['Stationer'] = merged_df['Stationer'].fillna(0)
merged_df

,Unnamed: 0,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Kommune,Rooms_imputed,Toilets_imputed,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand,Antal_laeger,Stationer
0,"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk ...",0.391,NaN,NaN,Rækkehus,D,Høj,24.60,25.00,1.0,128.0,388.0,2889.0,5.0,1947,1.0,2600,https://www.boligsiden.dk/addresses/0a3f50a4-b...,0,0,0,glostrup kommune,0,0,2660,1096,407100.0,47.4,23899.0,0.738859,0.055232,0.205908,13.3,6.0,0.0
1,"Strandpromenaden 21, 1. tv.,3000",6095000.0,Skolen i Bymidten,0.336,NaN,NaN,Ejerlejlighed,A2015,Lav,25.52,28.50,0.0,136.0,136.0,6056.0,5.0,2020,2.0,3000,https://www.boligsiden.dk/addresses/0d5cec39-0...,0,0,0,helsingør kommune,0,0,5089,1787,463089.0,58.8,64016.0,0.857786,0.045442,0.096773,23.6,11.0,1.0
2,"Hjortdalvej 331,9460",3995000.0,Skovsgård Tranum skole,5.240,6.9,3.6,Landejendom,D,Meget lav,25.70,32.85,0.0,173.0,173706.0,2089.0,5.0,1912,1.0,9460,https://www.boligsiden.dk/addresses/39ff92cd-f...,0,0,0,jammerbugt kommune,0,0,1421,537,409908.0,136.7,38170.0,0.928818,0.032119,0.039062,29.5,2.0,0.0
3,"Lystgårdsparken 34,8300",2798000.0,Parkvejens Skole,5.160,7.0,3.7,Villa,B,Mellem,25.10,30.32,180.0,159.0,758.0,1838.0,4.0,2008,1.0,8300,https://www.boligsiden.dk/addresses/363cc2db-6...,0,0,0,odder kommune,0,0,839,163,459467.0,70.8,23971.0,0.912770,0.036085,0.051145,23.6,10.0,0.0
4,"Fuglsøvej 92,8420",6000000.0,Molsskolen,3.740,7.4,3.7,Villa,D,Høj,25.90,33.80,0.0,265.0,8889.0,3434.0,12.0,1884,4.0,8420,https://www.boligsiden.dk/addresses/0a3f50bf-6...,0,0,0,syddjurs kommune,0,0,2009,647,432820.0,105.3,44033.0,0.916472,0.042582,0.040947,31.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19846,"Trongårdsparken 91,2800",10495000.0,Trongårdsskolen,0.242,8.0,3.8,Villa,D,Lav,24.38,21.49,109.0,184.0,1234.0,7030.0,6.0,1963,3.0,2800,https://www.boligsiden.dk/addresses/0a3f50a6-6...,0,0,0,lyngby-taarbæk kommune,0,0,4445,1234,576034.0,39.9,58530.0,0.808474,0.072612,0.118913,14.8,13.0,2.0
19847,"Alpinavej 16,8660",3995000.0,Stilling Skole,0.642,7.5,3.7,Rækkehus,C,Mellem,26.00,24.66,505.0,126.0,490.0,3253.0,4.0,1978,2.0,8660,https://www.boligsiden.dk/addresses/0a3f50c2-2...,0,0,0,skanderborg kommune,0,0,2406,774,528174.0,67.6,65376.0,0.920277,0.027365,0.052359,25.6,7.0,1.0
19848,"Thorsvej 39,9700",2375000.0,Skoledistrikt Vest,0.708,NaN,NaN,Villa,D,Mellem,26.30,31.23,290.0,186.0,1064.0,2230.0,7.0,1978,2.0,9700,https://www.boligsiden.dk/addresses/0a3f50c6-f...,0,0,0,brønderslev kommune,0,0,1720,933,410545.0,87.2,36602.0,0.916808,0.029097,0.054095,29.4,7.0,1.0
19849,"Strandjægervej 21,3630",4498000.0,Jægerspris Skole,10.380,6.8,3.6,Villa,A2010,Mellem,25.80,32.50,110.0,322.0,3579.0,4805.0,8.0,1787,2.0,3630,https://www.boligsiden.dk/addresses/0a3f50aa-4...,0,0,0,frederikssund kommune,0,0,2605,905,451448.0,51.8,46856.0,0.896683,0.035470,0.067846,24.7,1.0,0.0


In [16]:
merged_df.to_csv('merged_df_26_aug_Mie.csv')

Undersøger postnumre uden stationer og praktiserende læger

In [16]:
#merged_missings_df = merged_df[(merged_df['Antal_laeger']==0) |( merged_df['Stationer']==0)]
#merged_missings_df.to_csv('CSV_output/missings_station_læge.csv')

Herfra og ned er der kun rod 


.

In [10]:
#DF med postnumre:
postnummer_by_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
postnummer_by_df = postnummer_by_df.iloc[:, 2]
#Split kolonnen til 'Postal code', 'By_fra_DST', 'Kommune_fra_DST'
extracted_data = postnummer_by_df.str.extract(r'(\d+)\s+(.+)\s\((.+)\)')
extracted_data.columns = ['Postal code', 'By_fra_DST', 'Kommune_fra_DST']
extracted_data = extracted_data.dropna(how='all')
extracted_data['Postal code'] = extracted_data['Postal code'].astype('int32') 
postnr_by_kommune = extracted_data


#Liste med postnumre enten uden by eller læge
merged_1 = pd.merge(merged_df, laeger_st_post_df, on = 'Postal code', how = 'left')
merged_1_missing = merged_1[(merged_1['Antal_laeger'].isna()) | (merged_1['Stationer'].isna())]                                

KeyError: 'Antal_laeger'

In [ ]:
merged_1_missing.to_csv('CSV_output/postnumre_uden_læge_station.csv')

In [ ]:
merged_1_missing

In [ ]:
# MERGE POSTNUMRE OG BY PÅ DF_10000
postnummer_by_boligsiden_df = pd.merge(boligsiden_20000_df, postnr_by_kommune, on='Postal code', how = 'left') 
postnummer_by_boligsiden_df = postnummer_by_boligsiden_df.dropna(subset=['Price'])  #Så dropper vi de postnumre, hvor der ikke er noget til salg
print(len(postnummer_by_boligsiden_df))
print(len(boligsiden_20000_df))

In [ ]:
bolig_st_laeger_DST_df = pd.merge(postnummer_by_boligsiden_df, laeger_st_post_df, on = 'Postal code', how = 'left')
bolig_st_laeger_DST_df = bolig_st_laeger_DST_df.dropna(subset=['Price'])  #Så dropper vi de postnumre, hvor der ikke er noget til salg
print(len(bolig_st_laeger_DST_df))

In [ ]:
bolig_st_laeger_DST_df

In [ ]:
bolig_st_laeger_DST_df.to_csv('CSV_output/Mie_total_df.csv', index=False)

In [ ]:
bolig_st_laeger_DST_df_2 = pd.merge(bolig_st_laeger_DST_df, DST_df, on = 'By_fra_DST', how = 'left')
bolig_st_laeger_DST_df_2 = bolig_st_laeger_DST_df_2.dropna(subset=['Price'])
print(len(bolig_st_laeger_DST_df_2))
bolig_st_laeger_DST_df_2

In [ ]:
bolig_laeger_st_DST_df = bolig_st_laeger_DST_df_2

In [ ]:
bolig_laeger_st_DST_df = bolig_laeger_st_DST_df.dropna(subset=['Price'])

In [ ]:
bolig_laeger_st_DST_df.to_csv('CSV_output/bolig_laeger_st_DST.csv', index=False) 

In [ ]:
len(bolig_laeger_st_DST_df)

In [ ]:
grouped_df = bolig_laeger_st_DST_df.groupby('Municipiality')['antal anmeldelser'].mean()



In [ ]:
DST_df = pd.read_csv('CSV_output/DST.csv')
laeger_st_post_df = pd.read_csv('CSV_output/laeger_stationer_post.csv')
postnummer_by_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
boligsiden_10000_df = pd.read_csv('CSV_input/boligsiden_10000.csv')
boligsiden_20000_df 
boligsiden_20000_df

In [ ]:



# Load the two DataFrames from the provided CSV files
df_final_1123 = pd.read_csv('CSV_input/df_final_1123.csv')
DST_df = pd.read_csv('CSV_output/DST.csv')

# Rename the 'Kommune' column in df_final_1123 to match the 'Municipality' column in DST_df
df_final_1123 = df_final_1123.rename(columns={'Kommune': 'Municipality'})

# Merge the DataFrames on the 'Municipality' column, keeping all rows from df_final_1123
merged_df = pd.merge(df_final_1123, DST_df, on='Municipality', how='left')

# Display the first few rows of the merged DataFrame to confirm
print(merged_df.head())


In [ ]:
# Load the two DataFrames from the provided CSV files
df_final_1123 = pd.read_csv('CSV_input/df_final_1123.csv')
DST_df = pd.read_csv('CSV_output/DST.csv')


# Standardize the municipality names in both DataFrames
df_final_1123['Municipiality'] = df_final_1123['Municipiality'].str.strip().str.lower()
DST_df['Kommune'] = DST_df['Kommune'].str.strip().str.lower()

# Merge the DataFrames on the standardized 'municipality' column from df_final_1123 and 'kommune' from DST_df
merged_df = pd.merge(df_final_1123, DST_df, left_on='municipiality', right_on='kommune', how='left')

# Check for municipalities in df_final_1123 that didn't match with any in DST_df
unmatched_municipalities = df_final_1123[~df_final_1123['municipiality'].isin(DST_df['kommune'])]

# Save the merged DataFrame and the unmatched municipalities to new CSV files
#merged_file_path = '/mnt/data/merged_df_final_standardized.csv'
#unmatched_file_path = '/mnt/data/unmatched_municipalities.csv'

#merged_df.to_csv(merged_file_path, index=False)
#unmatched_municipalities.to_csv(unmatched_file_path, index=False)